This is a test of the file format `parquet`. The goal is to preprocess data once and then store it efficiently. Parquet allows for efficient read and writing as well as metadata for storing information about the preprocessing steps.

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from scripts import *

Data utils loaded


In [6]:
# Threshold (%) of data for a feature to be included
THRESHOLD = 0.75
# Depth Threshold (%) of data included from layer data.
# --- picks the maximum depth that includes the threshold % of devices
DEPTH_THRESHOLD = 0.75

EXCLUDED_SECTIONS = [
    "Reference information",
    "Cell definition",
    "Outdoor testing",
    # "Additional layers",
    "JV data"
]

EXCLUDED_COLS = [
    "Outdoor_time_start",
    "Outdoor_time_end"
]

In [7]:
data, selector = preprocess_data(
    THRESHOLD,
    DEPTH_THRESHOLD,
    exclude_sections=EXCLUDED_SECTIONS,
    exclude_cols=EXCLUDED_COLS,
    verbose=False
)

c:\Users\Violet\Documents\GitHub\PerovskiteML\src\notebooks
Data Initialized.
Preprocessing Data...
Threshold: 0.75, Depth: 0.75
Data Preprocessed.


c:\Users\Violet\Documents\GitHub\PerovskiteML\src\data_utils\__init__.py:96: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.to_numeric(col, errors='ignore')


In [8]:
data

,Substrate_stack_sequence_Layer_0,Substrate_stack_sequence_Layer_1,ETL_stack_sequence_Layer_0,ETL_stack_sequence_Layer_1,ETL_deposition_procedure_Layer_0,ETL_deposition_procedure_Layer_1,Perovskite_dimension_list_of_layers,Perovskite_composition_a_ions_Feature_0,Perovskite_composition_a_ions_Feature_1,Perovskite_composition_a_ions_coefficients_Feature_0,...,Add_lay_back,Encapsulation,Stabilised_performance_measured,EQE_measured,Stability_measured,Stability_average_over_n_number_of_cells,Stability_light_UV_filter,Stability_periodic_JV_measurements,Stability_PCE_burn_in_observed,Stability_flexible_cell_number_of_bending_cycles
0,SLG,FTO,TiO2-c,TiO2-mp,Spray-pyrolys,Spin-coating,3.0,Cs,None,1,...,False,False,False,False,False,1.0,False,False,False,0.0
1,SLG,FTO,TiO2-c,TiO2-mp,Spray-pyrolys,Spin-coating,3.0,Cs,None,1,...,False,False,False,False,False,1.0,False,False,False,0.0
2,SLG,FTO,TiO2-c,TiO2-mp,Spray-pyrolys,Spin-coating,3.0,Cs,None,1,...,False,False,False,False,False,1.0,False,False,False,0.0
3,SLG,FTO,TiO2-c,TiO2-mp,Spray-pyrolys,Spin-coating,3.0,Cs,None,1,...,False,False,False,False,False,1.0,False,False,False,0.0
4,SLG,FTO,TiO2-c,TiO2-mp,Spray-pyrolys,Spin-coating,3.0,Cs,None,1,...,False,False,False,False,False,1.0,False,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43247,SLG,ITO,ICBA,BCP,Spin-coating,Spin-coating,NaN,(PEA),FA,0.08,...,False,False,False,False,False,1.0,False,False,False,0.0
43248,SLG,ITO,ICBA,BCP,Spin-coating,Spin-coating,NaN,(PEA),FA,0.08,...,False,False,False,False,False,1.0,False,False,False,0.0
43249,SLG,ITO,ICBA,BCP,Spin-coating,Spin-coating,NaN,(PEA),FA,0.08,...,False,False,False,False,False,1.0,False,False,False,0.0
43250,SLG,FTO,TiO2-c,TiO2-mp,Spray-pyrolys,Spin-coating,3.0,MA,None,1,...,False,True,False,False,True,1.0,False,False,False,0.0


In [19]:
hyperparameters = {
    'threshold': THRESHOLD,
    'depth': DEPTH_THRESHOLD,
    'xsections': EXCLUDED_SECTIONS,
    'xcols': EXCLUDED_COLS
}
table = pa.Table.from_pandas(data)
metadata = table.schema.metadata or {}
metadata.update({key.encode(): str(value).encode() for key, value in hyperparameters.items()})
print(metadata)
pq.write_table(table.replace_schema_metadata(metadata), 'preprocessed_data.parquet')


{b'pandas': b'{"index_columns": [{"kind": "range", "name": null, "start": 0, "stop": 43252, "step": 1}], "column_indexes": [{"name": null, "field_name": null, "pandas_type": "unicode", "numpy_type": "object", "metadata": {"encoding": "UTF-8"}}], "columns": [{"name": "Substrate_stack_sequence_Layer_0", "field_name": "Substrate_stack_sequence_Layer_0", "pandas_type": "unicode", "numpy_type": "object", "metadata": null}, {"name": "Substrate_stack_sequence_Layer_1", "field_name": "Substrate_stack_sequence_Layer_1", "pandas_type": "unicode", "numpy_type": "object", "metadata": null}, {"name": "ETL_stack_sequence_Layer_0", "field_name": "ETL_stack_sequence_Layer_0", "pandas_type": "unicode", "numpy_type": "object", "metadata": null}, {"name": "ETL_stack_sequence_Layer_1", "field_name": "ETL_stack_sequence_Layer_1", "pandas_type": "unicode", "numpy_type": "object", "metadata": null}, {"name": "ETL_deposition_procedure_Layer_0", "field_name": "ETL_deposition_procedure_Layer_0", "pandas_type": 

In [21]:
pq.write_table(table.replace_schema_metadata(metadata), '../../data/preprocess/preprocessed_data.parquet')